# Fine tuning 3 resnet 34 fusion model 

In [1]:
import os
import sys
import glob
import shutil
import random
import pickle
import numpy as np
from PIL import Image
import time
import copy
from tqdm import tqdm

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
import torchvision.transforms as transforms
import torch.utils.data
from torchvision.models import resnet34
from torchvision import datasets, models, transforms
import torch.optim as optim
from torch.optim import lr_scheduler



# Add the src directory for functions
src_dir = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'src')
print(src_dir)
sys.path.append(src_dir)

# import my functions:
from functions import*

# Set it to use GPU1
torch.cuda.set_device(1)
print(torch.cuda.is_available())
print(torch.cuda.current_device())

/media/rene/Data/idc/src
True
1


In [2]:
# def make_fusion_model(model_list, model_name_list, save_path, input_size, model=WeightedSum):
#     """
#     model_list: list of model architectures
#     model_name_list: list of the names files of model weights
    
#     """
#     model_list_ft = []    
#     for idx, model_arch in enumerate(model_list):
#         # get the proper model architecture
#         model_ft = model_arch(pretrained=False)
#         num_ftrs = model_ft.fc.in_features
#         model_ft.fc = nn.Linear(num_ftrs, 2)
#         model_ft = model_ft.cuda()
#         # load the saved weights
#         model_ft.load_state_dict(torch.load(os.path.join(save_path, model_name_list[idx])))
#         model_list_ft.append(model_ft)
        
#     fusion_model = model(num_input=6)
#     fusion_model = fusion_model.cuda()
#     return model_list_ft, fusion_model

In [2]:
class Fusion1(nn.Module):
    """Take list of models, fuse their output into 2 classes"""
    def __init__(self, model_list):
        super(Fusion1, self).__init__()
        self.model_list = model_list
        self.num_input = int(len(self.model_list)*2)
        self.fc = nn.Linear(self.num_input, 2)

    def forward(self, x):
        outputs = []
        for model in model_list:
            outputs.append(model(x))
        x = torch.cat(outputs, 1)
        out = self.fc(x)
        return out
    
class Fusion1Hardcode(nn.Module):
    """Take list of models, fuse their output into 2 classes"""
    def __init__(self, model_list):
        super(Fusion1Hardcode, self).__init__()
        self.model1 = model_list[0]
        self.model2 = model_list[1]
        self.model3 = model_list[2]
        self.fc = nn.Linear(6, 2)

    def forward(self, x):
        x1 = self.model1(x)
        x2 = self.model2(x)
        x3 = self.model3(x)
        x4 = torch.cat((x1, x2, x3), 1)
        out = self.fc(x4)
        return out

In [8]:
# class Fusion2(nn.Module):
#     """FusionModel1 plus extra fc layer after the concat so there is non-linear combination of the outputs 
#     Take list of models, concat output->16 node fc->output(2 class)
#     """
#     def __init__(self, model_list):
#         super(Fusion1, self).__init__()
#         self.model_list = model_list
#         self.num_input = int(len(self.model_list)*2)
#         self.fc1 = nn.Linear(self.num_input, 16)
#         self.fc2 = nn.Linear(16, 2)
        
#     def forward(self, x):
#         outputs = []
#         for model in model_list:
#             outputs.append(model(x))
#         x1 = torch.cat(outputs, 1)
#         x2 = nn.ReLU(self.fc1(x))
#         out = self.fc2(x2)
#         return out
    
class Fusion2Hardcode(nn.Module):
    """Take list of models, fuse their output into 2 classes"""
    def __init__(self, model_list):
        super(Fusion2Hardcode, self).__init__()
        self.model1 = model_list[0]
        self.model2 = model_list[1]
        self.model3 = model_list[2]
        self.fc1 = nn.Linear(6, 16)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(16, 2)
        
    def forward(self, x):
        x1 = self.model1(x)
        x2 = self.model2(x)
        x3 = self.model3(x)
        x4 = torch.cat((x1, x2, x3), 1)
        x5 = self.fc1(x4)
        x6 = self.relu(x5)
        out = self.fc2(x5)
        return out

In [4]:
class Fusion3(nn.Module):
    """Throw away final classification layer before the concat is done.
    More parameters are in merged section of model: moving towards to a single model 
    Pop last (classification) layer of models, concat output->256 node fc->output(2 class)
    """
    def __init__(self, model_list):
        super(Fusion1, self).__init__()
        new_models = []
        model_ft.fc.in_features = 0
        for model in new_models:
            model_ft.fc.in_features += model.fc.in_features
            modules = list(model.children())[:-1]      # delete the last fc layer.
            model = nn.Sequential(*modules)
            new_models.append(model)        
        
        self.model_list = new_models
        self.num_input = int(len(self.model_list)*2)
        self.fc1 = nn.Linear(num_input, 256)
        self.fc2 = nn.Linear(256, 2)
        
    def forward(self, x):
        outputs = []
        for model in model_list:
            outputs.append(model(x))
        x1 = torch.cat(outputs, 1)
        x2 = F.relu(self.fc1(x))
        out = self.fc2(x2)
        return out

In [7]:
### CHeck the model arch that the last layer is actually fc, not a sigmoid

## Train Fusion1

In [13]:
model_list = [resnet34, resnet34, resnet34]
model_name_list = ['resnet34_0', 'resnet34_5', 'resnet34_2']

batch_size = 16
num_workers = 6
save_path = '/media/rene/Data/data/idc/models'
PATH = '/media/rene/Data/data/idc'
input_size = 6

dataloaders, dataset_sizes = make_batch_gen(PATH, batch_size, num_workers, 
                                            valid_name='valid', test_name='test', size=197)

pretrained_model_list = []
for idx, model_arch in enumerate(model_list):
    model_ft = model_arch(pretrained=False)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, 2)
    model_ft = model_ft.cuda()
    model_ft.load_state_dict(torch.load(os.path.join(save_path, model_name_list[idx])))
    pretrained_model_list.append(model_ft)

In [ ]:
%%capture out1
epochs = 12
model_ft = Fusion1Hardcode(pretrained_model_list)
model_ft = model_ft.cuda()
model_name = 'Fusion1_1'

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)

best_acc, model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                                 epochs, dataloaders, dataset_sizes)
torch.save(model_ft.state_dict(), os.path.join(save_path, model_name))

In [ ]:
out1.show()

In [ ]:
%%capture cell_output
epochs = 12
model_ft = Fusion2Hardcode(pretrained_model_list)
model_ft = model_ft.cuda()
model_name = 'Fusion2_1'

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)

best_acc, model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                                 epochs, dataloaders, dataset_sizes)
torch.save(model_ft.state_dict(), os.path.join(save_path, model_name))

Epoch 0/11
----------


In [12]:
cell_output.show()

NameError: name 'cell_output' is not defined

## Pre-trained
* Cat ouptut features and add a fc layer onto the final classification layer, train all the weights

## Pre-trained
* pop the final classification layer, add in cat and output layer